In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
from transformers import BertConfig
from transformers import BertModel
import transformers
from sklearn.model_selection import train_test_split
import copy

from PIL import Image
import os
import json

import torch.nn as nn
from torchvision import transforms
from torchvision import models
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/MMHS150K.zip -d /content/MMHS150k/

Streaming output truncated to the last 5000 lines.
 extracting: /content/MMHS150k/img_txt/1113980007095861248.json  
  inflating: /content/MMHS150k/img_txt/1044652740092334080.json  
  inflating: /content/MMHS150k/img_txt/1035844636902924288.json  
 extracting: /content/MMHS150k/img_txt/1109520190465863681.json  
  inflating: /content/MMHS150k/img_txt/1061758040641363968.json  
  inflating: /content/MMHS150k/img_txt/1107740216918458368.json  
 extracting: /content/MMHS150k/img_txt/1056521471210807296.json  
 extracting: /content/MMHS150k/img_txt/1110015616575041536.json  
  inflating: /content/MMHS150k/img_txt/1115195274748727297.json  
  inflating: /content/MMHS150k/img_txt/1057617106613747712.json  
  inflating: /content/MMHS150k/img_txt/1113868906538496000.json  
  inflating: /content/MMHS150k/img_txt/1060936979381071873.json  
  inflating: /content/MMHS150k/img_txt/1056417563310260224.json  
  inflating: /content/MMHS150k/img_txt/1108650061817237509.json  
  inflating: /content/MMH

In [ ]:
!pip install transformers torch torchvision sentencepiece
!pip install opencv-python

In [ ]:
DO_DEBUG_WITH_SMALL_DATASET = 0
NUM_OF_CLASSES = 6
probabilities = [0.6, 0.3, 0.1, 0.0, 0.0, 0.0]
mysoftmax=nn.Softmax(dim=0)
USELOCALTOKERN=1
#print(transformers.__version__  )
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(f"Using device: {device}")

In [ ]:
def compute_accuracy(taglist,predlist,itemlen=3):
    nTotal=len(taglist)
    nCorrect=0
    for k in range(nTotal):
        equal=0
        for i in range(itemlen):
            for j in range(itemlen):
                if taglist[k][i]==predlist[k][j]:
                    equal+=1
        if equal>=2:
            nCorrect+=1
    return 1.0*nCorrect / nTotal    #accuracy_score(taglist,predlist)
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient calculation
        for text_input_ids, text_attention_mask, image_inputs, labels in test_loader:

            preds0 = copy.deepcopy(labels)  # the labels is in shape as Tensor(batchsize,num_of_class)  batchsize=32,  num_of_class=6
            # to  get （First second third） data
            _, m1 = torch.max(preds0, dim=1)  # get max idx into m1
            # preds0[:,m1] = -1
            for k in range(len(m1)):
                preds0[k, m1[k].item()] = 0

            _, m2 = torch.max(preds0, dim=1)  # get the second max idx into m2
            # preds0[:,m2] = -1
            for k in range(len(m2)):
                preds0[k, m2[k].item()] = 0

            _, m3 = torch.max(preds0, dim=1)  # get the third max idx into m3
            # preds0[:,m3] = -1
            for k in range(len(m3)):
                preds0[k, m3[k].item()] = 0

            lable_Result = []
            for k in range(len(m3)):  # for each sample in the batch
                m = [m1[k].item(), m2[k].item(), m3[k].item()]
                # m.sort()                        # sort by idx number,for compyer_accrecy,no need to sort
                lable_Result.append(m)  # save the sorted label—index

            text_input_ids, text_attention_mask = text_input_ids.to(device), text_attention_mask.to(device)
            image_inputs, labels = image_inputs.to(device), labels.to(device)

            # Forward pass through the model
            outputs = model(text_input_ids, text_attention_mask, image_inputs)

            # Get the predicted class by taking the argmax of the outputs
            '''
            _, preds = torch.max(outputs, dim=1)
            _, labels = torch.max(labels, dim=1) # yg add
            # Store the labels and predictions
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())'''

            output = outputs.cpu()
            preds0 = copy.deepcopy(output)
            _, m1 = torch.max(preds0, dim=1)
            for k in range(len(m1)):
                preds0[k, m1[k].item()] = -1

            _, m2 = torch.max(preds0, dim=1)
            for k in range(len(m2)):
                preds0[k, m2[k].item()] = -1

            _, m3 = torch.max(preds0, dim=1)  # m3 = np.argmax(preds0)
            for k in range(len(m3)):
                preds0[k, m3[k].item()] = -1

            predRes = []
            for k in range(len(m3)):
                m = [m1[k].item(), m2[k].item(), m3[k].item()]
                # m.sort()
                predRes.append(m)

            # lable_Result=torch.tensor(lable_Result)
            # predRes=torch.tensor(predRes)
            all_labels.extend(lable_Result)
            all_preds.extend(predRes)

    # Calculate accuracy
    #accuracy = accuracy_score(all_labels, all_preds)
    accuracy = compute_accuracy(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy
def view_classification_results_2(model, test_loader, num_examples=10):
    model.eval()  # Set the model to evaluation mode
    examples_shown = 0

    with torch.no_grad():  # Disable gradient calculation
        for text_input_ids, text_attention_mask, image_inputs, labels in test_loader:

            text_input_ids, text_attention_mask = text_input_ids.to(device), text_attention_mask.to(device)
            image_inputs, labels = image_inputs.to(device), labels.to(device)
            # Forward pass through the model
            outputs = model(text_input_ids, text_attention_mask, image_inputs)

            # Get the predicted class by taking the argmax of the outputs
            #_, preds = torch.max(outputs, dim=1)

            # Loop through the batch and print predictions along with true labels
            if examples_shown >= num_examples:
                return  # Stop after showing the desired number of examples

            preds0=outputs.cpu().numpy()

            m1= np.argmax(preds0)
            preds0[m1]=-1

            m2 = np.argmax(preds0)
            preds0[m2] = -1

            m3 = np.argmax(preds0)
            preds0[m3] = -1

            preds = [m1,m2,m3]

            true_label = labels
            predicted_label =preds

            print(f"Example {examples_shown + 1}:")

            for i in range(len(preds)):
                print(f"Predicted Label: {predicted_label[i]}")
                print(f"True Label: {true_label[i]}")
                print("")

            examples_shown += 1



def view_classification_results(model, test_loader, num_examples=10):
    model.eval()  # Set the model to evaluation mode
    examples_shown = 0

    with torch.no_grad():  # Disable gradient calculation
        for text_input_ids, text_attention_mask, image_inputs, labels in test_loader:

            text_input_ids, text_attention_mask = text_input_ids.to(device), text_attention_mask.to(device)
            image_inputs, labels = image_inputs.to(device), labels.to(device)
            # Forward pass through the model
            outputs = model(text_input_ids, text_attention_mask, image_inputs)

            # Get the predicted class by taking the argmax of the outputs
            _, preds = torch.max(outputs, dim=1)

            # Loop through the batch and print predictions along with true labels
            for i in range(len(preds)):
                if examples_shown >= num_examples:
                    return  # Stop after showing the desired number of examples

                # true_label = labels[i].item()  # here  labels[i] == [ 0,0.3,0.1,0,0.6,0]
                # predicted_label = preds[i].item()

                #print("labels={}".format(labels[i]))
                #print("preds={}".format(outputs[i]))
                true_labels0 = labels[i].cpu() .detach().numpy()
                predicted_labels0 = outputs[i].cpu() .detach().numpy()    #_,=torch.max(outputs[i])
                min_label = np.min(true_labels0)
                min_pres_label = np.min(predicted_labels0)

                true_labels=[]
                predicted_labels=[]

                # add in first three label
                true_label= np.argmax(true_labels0)
                true_labels.append(true_label)
                true_labels0[true_label]=min_label
                true_label = np.argmax(true_labels0)
                true_labels.append(true_label)
                true_labels0[true_label] = min_label
                true_label = np.argmax(true_labels0)
                true_labels.append(true_label)


                predicted_label = np.argmax(predicted_labels0)
                predicted_labels.append(predicted_label)
                predicted_labels0[predicted_label]=min_pres_label
                predicted_label = np.argmax(predicted_labels0)
                predicted_labels.append(predicted_label)
                predicted_labels0[predicted_label] = min_pres_label
                predicted_label = np.argmax(predicted_labels0)
                predicted_labels.append(predicted_label)

                print(f"Example {examples_shown + 1}:")
                print(f"Predicted Label: {predicted_labels}")
                print(f"True Label: {true_labels}")
                print("")

                examples_shown += 1

In [ ]:
class MultimodalModel(nn.Module):
    def __init__(self):
        super(MultimodalModel, self).__init__()

        # Load pre-trained BERT model for text classification
        self.text_model = BertModel.from_pretrained('bert-base-uncased').to(device)

        # Load pre-trained ResNet model for image classification (output 512 features)
        self.image_model = models.resnet18(pretrained=True)
        self.image_model.fc = nn.Linear(self.image_model.fc.in_features, 512)  # Modify final layer
        self.image_model = self.image_model.to(device)

        # Final classification layer (512 + 768 from text BERT output)
        self.classifier = nn.Linear(512 + 768, 6).to(device)  # 6 output classes (NotHate, Racist, etc.)


    def forward(self, text_input_ids, text_attention_mask, image_input):
        # Forward pass through BERT for text
        text_outputs = self.text_model(input_ids=text_input_ids, attention_mask=text_attention_mask)
        text_embedding = text_outputs.pooler_output  # Use the [CLS] token output

        # Forward pass through ResNet for image
        image_embedding = self.image_model(image_input)

        # Concatenate text and image embeddings
        combined_embedding = torch.cat((text_embedding, image_embedding), dim=1)

        # Final classification output
        output = self.classifier(combined_embedding)
        output2 = mysoftmax(output)  # ygyg add in softmax
        return output2


In [ ]:
def evaluate_model(model, val_loader):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_preds = []
    first=1
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for text_input_ids, text_attention_mask, image_inputs, labels in val_loader:

            preds0 = copy.deepcopy(labels)      # the labels is in shape as Tensor(batchsize,num_of_class)  batchsize=32,  num_of_class=6
            # to  get （First second third） data
            _, m1 = torch.max(preds0, dim=1)    # get max idx into m1
            #preds0[:,m1] = -1
            for k in range(len(m1)):
                preds0[k,m1[k].item()] = 0

            _, m2 = torch.max(preds0, dim=1)    # get the second max idx into m2
            #preds0[:,m2] = -1
            for k in range(len(m2)):
                preds0[k, m2[k].item()] = 0

            _, m3 = torch.max(preds0, dim=1)    # get the third max idx into m3
            #preds0[:,m3] = -1
            for k in range(len(m3)):
                preds0[k, m3[k].item()] = 0

            lable_Result = []
            for k in range(len(m3)):            # for each sample in the batch
                m=[m1[k].item(),m2[k].item(),m3[k].item()]
                #m.sort()                        # sort by idx number,for compyer_accrecy,no need to sort
                lable_Result.append(m)          # save the sorted label—index

            # predict by model
            text_input_ids, text_attention_mask = text_input_ids.to(device), text_attention_mask.to(device)
            image_inputs, labels = image_inputs.to(device), labels.to(device)

            # Forward pass through the model
            outputs = model(text_input_ids, text_attention_mask, image_inputs)

            # Get the predicted class by taking the argmax of the outputs

            #outputs = outputs.cpu().numpy()
            '''
            _,preds_idx = torch.max(outputs, dim=1)  # pred is [batchsize][1]
            _,labels_idx = torch.max(labels,dim=1)
            all_labels.extend(labels_idx.cpu().numpy())
            all_preds.extend(preds_idx.cpu().numpy())'''


            output = outputs.cpu()
            preds0 = copy.deepcopy(output)
            _, m1 = torch.max(preds0, dim=1)
            for k in range(len(m1)):
                preds0[k, m1[k].item()] = -1

            _, m2 = torch.max(preds0, dim=1)
            for k in range(len(m2)):
                preds0[k, m2[k].item()] = -1

            _, m3 = torch.max(preds0, dim=1)    #m3 = np.argmax(preds0)
            for k in range(len(m3)):
                preds0[k, m3[k].item()] = -1

            predRes = []
            for k in range(len(m3)):
                m=[m1[k].item(),m2[k].item(),m3[k].item()]
                #m.sort()
                predRes.append(m)

            #lable_Result=torch.tensor(lable_Result)
            #predRes=torch.tensor(predRes)
            all_labels.extend(lable_Result)
            all_preds.extend(predRes)

    # Calculate accuracy
    #accuracy = accuracy_score(all_labels, all_preds)
    accuracy= compute_accuracy(all_labels, all_preds)
    return accuracy
# Training loop with validati
# on after each epoch
def train_and_validate(model, train_loader, val_loader, num_epochs=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        first=1
        for text_input_ids, text_attention_mask, image_inputs, labels in train_loader:

            optimizer.zero_grad()  # Clear previous gradients

            # labels=labels.softmax(dim=1)    # yg add in,no need,have done in __getitem__

            text_input_ids, text_attention_mask = text_input_ids.to(device), text_attention_mask.to(device)
            image_inputs, labels = image_inputs.to(device), labels.to(device)

            # Forward pass through the model
            outputs = model(text_input_ids, text_attention_mask, image_inputs)

            if first==1:
                #print("epoch ={},output={} \nlabels={}".format(epoch,outputs,labels))
                first = 0

            loss = criterion(outputs, labels)  # Compute the loss

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # After each epoch, validate the model
        val_accuracy = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        #print("GPU usage after epoch:")
        #!nvidia-smi


In [ ]:
class TweetDataset(Dataset):
    def __init__(self, tweet_data, tokenizer, image_transform):
        self.tweet_data = tweet_data  # List of dictionaries containing tweet information
        self.tokenizer = tokenizer    # BERT tokenizer for processing tweet text
        self.image_transform = image_transform  # Transformations for image preprocessing

    def __len__(self):
        return len(self.tweet_data)

    def __getitem__(self, idx):                 # Get tweet data at the given index
        tweet = self.tweet_data[idx]            # Process the text using the BERT tokenizer
        text = tweet['text']
        text_encoding = self.tokenizer(
            text, padding='max_length', truncation=True, max_length=128, return_tensors='pt'
        )
        text_input_ids = text_encoding['input_ids'].squeeze(0)  # Tokenized text input
        text_attention_mask = text_encoding['attention_mask'].squeeze(0)  # Attention mask

        # Process the image
        image = Image.open(tweet['img_path']).convert('RGB')  # Open image as a PIL Image
        image = self.image_transform(image)  # Apply transformations to convert to tensor
        # Get the label (use majority voting if needed)
        #label = max(set(tweet['labels']), key=tweet['labels'].count)  # Majority vote
        labels = tweet['labels']
        labels_out=[0.0] * NUM_OF_CLASSES
        '''  a method to count labels
        k=0
        for l in labels:
            if labels_out[l]==0.0:
                labels_out[l]=probabilities[k]
                k+=1'''
        for idx in labels:      # another method to count the labels
            labels_out[idx]+=1.0/3
        label = np.array(labels_out)
        label = mysoftmax(torch.tensor(label))
        return text_input_ids, text_attention_mask, image, label #  torch.tensor(label)


In [ ]:
def move_to_device(batch, device):
    text_input_ids, text_attention_mask, image_inputs, labels = batch
    text_input_ids = text_input_ids.to(device)
    text_attention_mask = text_attention_mask.to(device)
    image_inputs = image_inputs.to(device)
    labels = labels.to(device)
    return text_input_ids, text_attention_mask, image_inputs, labels

In [ ]:
if __name__ == '__main__':
    #Load json file
    json_path = '/content/MMHS150k/MMHS150K_GT.json'
    with open(json_path, 'r') as f:
        data = json.load(f)

    image_folder = '/content/MMHS150k/img_resized/'

    #Construct the empty list for later data storage
    tweet_data = []
    #Extract tweet text, labels, and image paths from the JSON data
    for tweet_id, tweet_info in data.items():
        img_path = os.path.join(image_folder, f"{tweet_id}.jpg")
        tweet_text = tweet_info['tweet_text']
        labels = tweet_info['labels']  # We'll use majority voting later
        labels_str = tweet_info['labels_str']

        # Append the relevant information in a dictionary for each tweet

        tweet_data.append({
            'tweet_id': tweet_id,
            'img_path': img_path,
            'text': tweet_text,
            'labels': labels,
            'labels_str': labels_str
        })

    #Inspect the data
    print(f"Number of tweets processed: {len(tweet_data)}")
    print(tweet_data[0])

    # Load the tweet IDs for each split
    with open('MMHS150k/splits/train_ids.txt', 'r') as f:
        train_ids = [line.strip() for line in f.readlines()]

    with open('MMHS150k/splits/val_ids.txt', 'r') as f:
        val_ids = [line.strip() for line in f.readlines()]

    with open('MMHS150k/splits/test_ids.txt', 'r') as f:
        test_ids = [line.strip() for line in f.readlines()]

    if DO_DEBUG_WITH_SMALL_DATASET==1:
        train_ids=train_ids[:64]
        val_ids = val_ids[:32]
        test_ids =test_ids[:32]
        print("Number of training ids: ", len(train_ids))
        print("Number of validation ids: ", len(val_ids))

    # Now split the tweet_data list into train, val, and test based on these IDs
    train_data = [tweet for tweet in tweet_data if tweet['tweet_id'] in train_ids]
    val_data = [tweet for tweet in tweet_data if tweet['tweet_id'] in val_ids]
    test_data = [tweet for tweet in tweet_data if tweet['tweet_id'] in test_ids]

    # Verify the sizes of the splits
    print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}, Test samples: {len(test_data)}")

    # Define the image transformations (resize to 224x224 and normalize)
    image_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load the BERT tokenizer

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    '''
    if USELOCALTOKERN==0:
        tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased/')
        #tokenizer = BertTokenizer.from_pretrained('bert_pretrain')
    else:
        #config = BertConfig.from_json_file("bert-base-uncased/config.json")
        #model0 = BertModel.from_pretrained("bert-base-uncased/pytorch_model.bin", config=config)
        tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased/') '''

    if tokenizer:
        print('tokenizer init successfully!')
        print(tokenizer)
    else:
        print('tokenizer init failure!')
        assert (1==0)

    # Custom dataset class to handle the multimodal data

    # Create Dataset objects for each split
    train_dataset = TweetDataset(train_data, tokenizer, image_transform)
    val_dataset = TweetDataset(val_data, tokenizer, image_transform)
    test_dataset = TweetDataset(test_data, tokenizer, image_transform)

    num_workers = 10 #10
    batch_size = 32
    if DO_DEBUG_WITH_SMALL_DATASET==1:
        num_workers=1
    # Create DataLoader objects
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers, pin_memory=True)


    # Verify the DataLoader is working and moving data to device
    ''' for there is the test,no need to run
    for batch in train_loader:
        batch = move_to_device(batch, device)
    for batch in val_loader:
        batch = move_to_device(batch, device)
    for batch in test_loader:
        batch = move_to_device(batch, device)'''

    # Verify the DataLoader is working
    #print(next(iter(train_loader)))



    # Initialize the model, loss function, and optimizer
    model = MultimodalModel().to(device)
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4)


    # Call the training function
    num_epochs=5
    if DO_DEBUG_WITH_SMALL_DATASET==1:
        num_epochs=1
    train_and_validate(model, train_loader, val_loader, num_epochs)


    # Call the test function after training
    test_model(model, test_loader)


    # Example usage after training
    view_classification_results(model, test_loader, num_examples=100) # 100

Number of tweets processed: 149823
{'tweet_id': '1114679353714016256', 'img_path': '/content/MMHS150k/img_resized/1114679353714016256.jpg', 'text': '@FriskDontMiss Nigga https://t.co/cAsaLWEpue', 'labels': [4, 1, 3], 'labels_str': ['Religion', 'Racist', 'Homophobe']}
Training samples: 134823, Validation samples: 5000, Test samples: 10000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


tokenizer init successfully!
BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 87.0MB/s]
